# Aprimorando a MLP com Grid Search para escolher o melhor número de camadas intermediárias no Mapa Logístico

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [6]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import logisticmap as logmap

## 2. Gerando a série temporal do mapa logístico

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original. Utilizaremos  $r = 3.86$, o que faz com que o sistema esteja em caos.

In [7]:
x_inicial = 0.5
n_inicial = 0
n_iteracoes = 5000
r=3.86
estados = np.array([x_inicial, n_inicial])

In [8]:
mapa = logmap.MapaLogistico(estado_inicial=estados, r=r)

In [9]:
for n in range(0, n_iteracoes):
    mapa.iterar()
    estados = np.vstack((estados, mapa.ler_estado()))

In [10]:
x = estados[:, 0]
n = estados[:, 1]

In [11]:
fig, ax = plt.subplots()
ax.plot(n, x, color='Crimson')

ax.set_title("$100$ iterações iniciais da série temporal do Mapa Logístico\n para $r =$ " + str(r) + " com $x[0] =$ " + str(x[0]))
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(0,100)
    
ax.grid(True)
sns.despine()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [12]:
K = 4
L = 3
tam_teste = 0.15

In [13]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [14]:
serie_temporal.criar_matrizes()

In [15]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [16]:
def criar_modelo(input_size, optimizer='Nadam', batch_normalization='OFF', learning_rate=0.001, activation='selu', init_mode='lecun_normal', n_neurons=30,
                 n_hidden_layers=1, loss="mean_squared_error", name="MLP"):
    
    if (optimizer=='Nadam'):
        model_optimizer = keras.optimizers.Nadam()
    
    model_optimizer.learning_rate.assign(learning_rate)
    
    model = keras.Sequential(name=name)
    model.add(keras.layers.Dense(input_size, input_dim=input_size, name="camada_de_entrada", activation = 'linear'))
    if (batch_normalization == 'ON'):
        model.add(keras.layers.BatchNormalization(name="camada_de_batch_normalization"))
    for i in range(0, n_hidden_layers):
        model.add(keras.layers.Dense(n_neurons, input_dim=input_size, activation=activation, kernel_initializer=init_mode, name="camada_intermediaria_"+str(i+1)))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model.compile(
        optimizer = model_optimizer,
        loss = loss)
    
    model.build()
    return model

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [17]:
loss = "mean_squared_error"
optimizer = 'Nadam'
batch_size = 2
batch_normalization = 'OFF'
activation = 'tanh'
init_mode = 'glorot_uniform'
n_neurons = 10
learning_rate = 0.003
input_size = K

### 3.3 Executando *Grid Search* para saber o melhor número de camadas intermediárias

In [18]:
model_cv = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=batch_size, batch_normalization=batch_normalization,
                          learning_rate=learning_rate, activation=activation, init_mode=init_mode, n_neurons=n_neurons,
                          loss=loss, optimizer=optimizer, input_size=input_size)

In [19]:
param_grid = {'n_hidden_layers' : (1, 2, 3, 4, 5)}

In [20]:
grid = GridSearchCV(estimator=model_cv, param_grid=param_grid, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [21]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


KeyboardInterrupt: 

In [56]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000879 using {'n_hidden_layers': 3}


In [57]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.003577 (0.001562) with: {'n_hidden_layers': 1}
-0.002442 (0.002089) with: {'n_hidden_layers': 2}
-0.000879 (0.000299) with: {'n_hidden_layers': 3}
-0.001615 (0.001482) with: {'n_hidden_layers': 4}
-0.005203 (0.002802) with: {'n_hidden_layers': 5}


### Treino com o melhor modelo

In [ ]:
n_hidden_layers=

In [42]:
model = criar_modelo(name="MLP-Logistic", input_size=K, optimizer=optimizer, batch_normalization=batch_normalization, learning_rate=learning_rate, activation=activation,
                     init_mode=init_mode, n_neurons=n_neurons, n_hidden_layers=n_hidden_layers, loss=loss)

In [43]:
model.summary()

Model: "MLP-1-camada-intermediaria"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria (Dense) (None, 10)                50        
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 11        
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [44]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [45]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [47]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
1910/1910 [==============================] - 3s 1ms/step - loss: 0.0588 - val_loss: 0.0565
Epoch 2/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0509 - val_loss: 0.0546
Epoch 3/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0512 - val_loss: 0.0524
Epoch 4/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0493 - val_loss: 0.0536
Epoch 5/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0447 - val_loss: 0.0477
Epoch 6/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0414 - val_loss: 0.0460
Epoch 7/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0392 - val_loss: 0.0448
Epoch 8/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0369 - val_loss: 0.0420
Epoch 9/100
1910/1910 [==============================] - 2s 1ms/step - loss: 0.0344 - val_loss: 0.0378
Epoch 10/100
1910/1910 [==============================] - 2s 1ms/step - l

## Teste com o melhor modelo

In [48]:
y_pred = model.predict(X_teste)

### Gráfico

In [53]:
fig, ax = plt.subplots()
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_teste, color='Crimson', label='Valor real')
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title("Comparação da predição da MLP com o valor real do mapa logístico\n utilizando a rede neural ótima no conjunto de teste")
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(4900, 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-aprimorada/performance/mlp-aprimorada-vs-logistic.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Erro Quadrático Médio

In [1]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

### Salvando o Modelo

In [70]:
model.save("../../../../models/mlp-aprimorada/mlp-aprimorada-logistic.h5", include_optimizer=True)